In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 3

In [3]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_358885/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_358885/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.5944784539086478,
 'decoded_ood_sinkhorn_div_1': 118.47601754324776,
 'decoded_ood_sinkhorn_div_10': 86.2242202758789,
 'decoded_ood_sinkhorn_div_100': 40.72489493233817,
 'decoded_ood_e_distance': 80.45091606115287,
 'decoded_ood_mmd': 0.30328609687941416}

In [14]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7734585578242937,
 'decoded_test_e_distance': 43.814013458069645,
 'decoded_test_mmd_distance': 0.28129930421710014}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.5944784539086478,
 'decoded_ood_sinkhorn_div_1': 118.47601754324776,
 'decoded_ood_sinkhorn_div_10': 86.2242202758789,
 'decoded_ood_sinkhorn_div_100': 40.72489493233817,
 'decoded_ood_e_distance': 80.45091606115287,
 'decoded_ood_mmd': 0.30328609687941416}

In [16]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -13.451267182826996,
 'encoded_ood_sinkhorn_div_1': 38.241009541920256,
 'encoded_ood_sinkhorn_div_10': 34.273105825696675,
 'encoded_ood_sinkhorn_div_100': 32.81482015337263,
 'encoded_ood_e_distance': 65.27609379600499,
 'encoded_ood_mmd': 0.34757685235568453}

In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.24654958077839442,
 'deg_ood_sinkhorn_div_1': 34.06803635188511,
 'deg_ood_sinkhorn_div_10': 25.067314420427596,
 'deg_ood_sinkhorn_div_100': 24.3095828465053,
 'deg_ood_e_distance': 48.46170391712991,
 'deg_ood_mmd': 0.3940129961286272}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.6069448590278625,
 'deg_test_sinkhorn_div_1': 21.912623008092243,
 'deg_test_sinkhorn_div_10': 13.61953322092692,
 'deg_test_sinkhorn_div_100': 12.598226269086203,
 'deg_test_e_distance': 25.053625080741885,
 'deg_test_mmd': 0.3364326022565365}

In [19]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7734585578242937,
 'decoded_test_e_distance': 43.814013458069645,
 'decoded_test_mmd_distance': 0.28129930421710014}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.5944784539086478,
 'decoded_ood_sinkhorn_div_1': 118.47601754324776,
 'decoded_ood_sinkhorn_div_10': 86.2242202758789,
 'decoded_ood_sinkhorn_div_100': 40.72489493233817,
 'decoded_ood_e_distance': 80.45091606115287,
 'decoded_ood_mmd': 0.30328609687941416}

In [21]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.24654958077839442,
 'deg_ood_sinkhorn_div_1': 34.06803635188511,
 'deg_ood_sinkhorn_div_10': 25.067314420427596,
 'deg_ood_sinkhorn_div_100': 24.3095828465053,
 'deg_ood_e_distance': 48.46170391712991,
 'deg_ood_mmd': 0.3940129961286272}

In [22]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7734585578242937,
 'decoded_test_e_distance': 43.814013458069645,
 'decoded_test_mmd_distance': 0.28129930421710014}

In [23]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.19504256298822817,
 'encoded_test_sinkhorn_div_1': 25.463589012622833,
 'encoded_test_sinkhorn_div_10': 21.472211917241413,
 'encoded_test_sinkhorn_div_100': 20.207448919614155,
 'encoded_test_e_distance': 40.1504495237017,
 'encoded_test_mmd': 0.2532042060047388}

In [24]:
ood_metrics_encoded

{'Panobinostat+Crizotinib': {'r_squared': -7.258084297180176,
  'sinkhorn_div_1': 71.56011962890625,
  'sinkhorn_div_10': 65.60475158691406,
  'sinkhorn_div_100': 63.164188385009766,
  'e_distance': 125.6986238578861,
  'mmd': 0.46438327},
 'Panobinostat+Curcumin': {'r_squared': -20.296207427978516,
  'sinkhorn_div_1': 55.67168045043945,
  'sinkhorn_div_10': 50.19842529296875,
  'sinkhorn_div_100': 48.22564697265625,
  'e_distance': 95.9799371347084,
  'mmd': 0.43168664},
 'Panobinostat+SRT1720': {'r_squared': -54.46111297607422,
  'sinkhorn_div_1': 48.237247467041016,
  'sinkhorn_div_10': 42.964141845703125,
  'sinkhorn_div_100': 40.98801040649414,
  'e_distance': 81.52357354982553,
  'mmd': 0.40997002},
 'Panobinostat+Sorafenib': {'r_squared': -13.716713905334473,
  'sinkhorn_div_1': 62.6499137878418,
  'sinkhorn_div_10': 56.758811950683594,
  'sinkhorn_div_100': 54.484642028808594,
  'e_distance': 108.42152028383765,
  'mmd': 0.43778762},
 'SRT2104+Alvespimycin': {'r_squared': 0.958

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
